# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 12:40PM,258458,21,PO 1446729 GTS 641567 641668 TR#10648 TR#10649,"NBTY Global, Inc.",Released
1,Mar 1 2023 11:19AM,258453,10,MSP218990,"Methapharm, Inc.",Released
2,Mar 1 2023 11:19AM,258453,10,MSP218991,"Methapharm, Inc.",Released
3,Mar 1 2023 11:19AM,258453,10,MSP218992,"Methapharm, Inc.",Released
4,Mar 1 2023 11:19AM,258453,10,MSP218994,"Methapharm, Inc.",Released
5,Mar 1 2023 11:19AM,258453,10,MSP218995,"Methapharm, Inc.",Released
6,Mar 1 2023 11:19AM,258453,10,MSP218996,"Methapharm, Inc.",Released
7,Mar 1 2023 11:19AM,258453,10,MSP219031,"Methapharm, Inc.",Released
8,Mar 1 2023 11:19AM,258453,10,MSP219033,"Methapharm, Inc.",Released
9,Mar 1 2023 11:19AM,258453,10,MSP219025,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,258452,Executing,1
19,258452,Released,1
20,258453,Released,32
21,258454,Released,6
22,258458,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258451,1.0,NaN
258452,1.0,1.0
258453,NaN,32.0
258454,NaN,6.0
258458,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258375,27.0,29.0
258409,0.0,2.0
258417,0.0,1.0
258420,0.0,2.0
258421,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258375,27,29
258409,0,2
258417,0,1
258420,0,2
258421,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258375,27,29
1,258409,0,2
2,258417,0,1
3,258420,0,2
4,258421,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258375,27,29
1,258409,,2
2,258417,,1
3,258420,,2
4,258421,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 12:40PM,258458,21,"NBTY Global, Inc."
1,Mar 1 2023 11:19AM,258453,10,"Methapharm, Inc."
33,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc."
37,Mar 1 2023 11:18AM,258454,10,Methapharm-G
39,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc."
40,Mar 1 2023 11:00AM,258452,10,Emerginnova
42,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC"
43,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc."
44,Mar 1 2023 10:50AM,258446,10,Bio-PRF
58,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 1 2023 12:40PM,258458,21,"NBTY Global, Inc.",,1
1,Mar 1 2023 11:19AM,258453,10,"Methapharm, Inc.",,32
2,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",,6
3,Mar 1 2023 11:18AM,258454,10,Methapharm-G,,6
4,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",1,
5,Mar 1 2023 11:00AM,258452,10,Emerginnova,1,1
6,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",1,
7,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",,1
8,Mar 1 2023 10:50AM,258446,10,Bio-PRF,,14
9,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 12:40PM,258458,21,"NBTY Global, Inc.",1,
1,Mar 1 2023 11:19AM,258453,10,"Methapharm, Inc.",32,
2,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",6,
3,Mar 1 2023 11:18AM,258454,10,Methapharm-G,6,
4,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",,1
5,Mar 1 2023 11:00AM,258452,10,Emerginnova,1,1
6,Mar 1 2023 10:53AM,258449,10,"Zelenko Labs, LLC",,1
7,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",1,
8,Mar 1 2023 10:50AM,258446,10,Bio-PRF,14,
9,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 12:40PM,258458,21,"NBTY Global, Inc.",1,
1,Mar 1 2023 11:19AM,258453,10,"Methapharm, Inc.",32,
2,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",6,
3,Mar 1 2023 11:18AM,258454,10,Methapharm-G,6,
4,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 12:40PM,258458,21,"NBTY Global, Inc.",1.0,NaN
1,Mar 1 2023 11:19AM,258453,10,"Methapharm, Inc.",32.0,NaN
2,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",6.0,NaN
3,Mar 1 2023 11:18AM,258454,10,Methapharm-G,6.0,NaN
4,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 12:40PM,258458,21,"NBTY Global, Inc.",1.0,0.0
1,Mar 1 2023 11:19AM,258453,10,"Methapharm, Inc.",32.0,0.0
2,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",6.0,0.0
3,Mar 1 2023 11:18AM,258454,10,Methapharm-G,6.0,0.0
4,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2842896,165.0,8.0
12,258421,0.0,1.0
15,775224,32.0,27.0
19,775299,4.0,0.0
21,258458,1.0,0.0
22,516865,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2842896,165.0,8.0
1,12,258421,0.0,1.0
2,15,775224,32.0,27.0
3,19,775299,4.0,0.0
4,21,258458,1.0,0.0
5,22,516865,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,165.0,8.0
1,12,0.0,1.0
2,15,32.0,27.0
3,19,4.0,0.0
4,21,1.0,0.0
5,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,165.0
1,12,Released,0.0
2,15,Released,32.0
3,19,Released,4.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Executing,8.0,1.0,27.0,0.0,0.0,0.0
Released,165.0,0.0,32.0,4.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Executing,8.0,1.0,27.0,0.0,0.0,0.0
1,Released,165.0,0.0,32.0,4.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Executing,8.0,1.0,27.0,0.0,0.0,0.0
1,Released,165.0,0.0,32.0,4.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()